## Requirements

In [ ]:
import numpy as np 
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_column', None)

from sklearn.model_selection import train_test_split

#pre-processing libraries
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from string import punctuation
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
nltk.download('stopwords')

import sys
sys.path.insert(0,'/content/drive/MyDrive/')
from utils import process_tweet, build_freqs, abbreviation

!gdown --id 1jdI2jl2W-bFGWMfb59atn28o34GOF3ML
# import tweets.csv file 
!gdown --id 1NDH7awh6eYzxury-qkxAi0UIq6whOUih
 
# ELMo embeddings libraries
# !pip install tensorflow==1.15
# !pip install tensorflow_hub==0.6.0
# import tensorflow_hub as hub
# import tensorflow as tf
# elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

import pickle 
import pathlib
# bert embeddings
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer


# gensim libraries 
import gensim
from gensim.models import Word2Vec
length_w2v = 100

# TFIDF 
from sklearn.feature_extraction.text import TfidfVectorizer 

# # here we unzip the Glove dataset, with shape 300
# !unzip /content/drive/MyDrive/glove.6B.zip

# modeling libraries
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
import xgboost
from xgboost import XGBClassifier

# import torch
# import torch.nn as nn
# import torchvision
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader, TensorDataset
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# from collections import Counter   
# sequence_length = 40

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding,GRU, Dropout, MaxPooling1D
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix




import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')


Y_train = Y_test = X_train = X_test = None
embeddings = None

In [ ]:
def load_data():
  # all_positive_tweets = twitter_samples.strings('positive_tweets.json')
  # all_negative_tweets = twitter_samples.strings('negative_tweets.json')
  # test_pos = all_positive_tweets[4000:]
  # train_pos = all_positive_tweets[:4000]
  # test_neg = all_negative_tweets[4000:]
  # train_neg = all_negative_tweets[:4000]
  # X_train = train_pos + train_neg 
  # X_test = test_pos + test_neg
  # Y_train = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
  # Y_test = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)
  global Y_train, Y_test, X_train, X_test
  df = pd.read_csv("tweets.csv", encoding = "ISO-8859-1")
  df.columns = ["target", "id", "date", "flag", 'user', "tweet"]
  df = df.drop([ "id", "date", "flag", 'user'], axis=1)
  df = df.sample(frac=0.1, random_state=40)
  train_data, test_data = train_test_split(df, test_size=0.15,
                                         random_state=6)
  f = lambda x : 1 if x else 0
  train_data['target'] = train_data['target'].map(f)
  test_data['target'] = test_data['target'].map(f)
  X_train = train_data['tweet'].values.reshape(-1,1)
  X_test = test_data['tweet'].values.reshape(-1,1)
  Y_train = train_data['target'].values.reshape(-1,1)
  Y_test = test_data['target'].values.reshape(-1,1)
    
  # return X_train, X_test, Y_train, Y_test

In [ ]:
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    tweet = str(tweet)
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+', ' ', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    clean_tweets = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # clean_tweets.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            clean_tweets.append(stem_word)
    if len(clean_tweets)==0 :
      clean_tweets.append('UNK')
    return clean_tweets

### load embeddings if available

In [ ]:

# this code can be used when words embeddings are saved 



embeddings = pathlib.Path('/content/drive/MyDrive/embeddings.pickle')
embeddings_matrices = pathlib.Path('/content/drive/MyDrive/embeddings_matrices.pickle')

  # if files exists we load them instead of training from scratch 
if embeddings.is_file():
  # w2v, bert, tfidf and glove embeddings 
  pickle_in = open(embeddings, "rb")
  embeddings = pickle.load(pickle_in)

if embeddings_matrices.is_file():
  # w2v, bert, and glove embeddings_matrices 
  pickle_in = open(embeddings_matrices, "rb")
  embeddings_matrices = pickle.load(pickle_in)


#pre_processing 

In [ ]:
def get_processed_data(tokenized = True):
  load_data()
  global X_train, X_test
  # if(X_train != None and X_test != None):
  X_train = [process_tweet(X_train[i]) if tokenized  else  ' '.join(process_tweet(X_train[i])) for i in  range(len(X_train))]
  X_test = [process_tweet(X_test[i]) if tokenized  else  ' '.join(process_tweet(X_test[i])) for i in  range(len(X_test))]
  

## Bert 

In [ ]:
def get_bert():
  bert_model = SentenceTransformer('bert-base-nli-mean-tokens')
  # X_train, X_test = get_processed_data()
  sentences = ["[CLS] " + str(TWEET) + " [SEP]" for TWEET in X_train]
  X_train_bert = bert_model.encode(sentences)
  sentences = ["[CLS] " + str(TWEET) + " [SEP]" for TWEET in X_test]
  X_test_bert = bert_model.encode(sentences)
  return X_train_bert, X_test_bert


def get_bert_Seq(vocab):
  embedding_matrix = np.zeros((len(vocab)+1, 768))
  bert_model = SentenceTransformer('bert-base-nli-mean-tokens')
  words = list(vocab.keys())
  embedding_matrix[1:, :] = np.array(bert_model.encode(words))
  return embedding_matrix


##Word2Vec 

In [ ]:
def get_w2v():
  # get_processed_data()
  data = np.append(X_train, X_test)
  word2vec = Word2Vec(data, size=length_w2v, window=8, min_count=4)
  zeros = np.zeros(length_w2v)
  X_train_w2v = [ np.sum([ word2vec.wv[x] if x in word2vec.wv.vocab else zeros for x in X_train[i]  ], axis=0)/len(X_train[i]) for i in range(len(X_train))]
  X_test_w2v = [ np.sum([ word2vec.wv[x] if x in word2vec.wv.vocab else zeros for x in X_test[i]   ], axis=0)/len(X_test[i])  for i in range(len(X_test))]
  return np.array(X_train_w2v), np.array(X_test_w2v), word2vec


def get_w2v_seq(vocab):
  word2vec = Word2Vec(X_train, size=length_w2v, window=5, min_count=4)
  embedding_matrix = np.zeros((len(vocab)+1, length_w2v))
  for word, token in vocab.items():
      if word2vec.wv.__contains__(word):
          embedding_matrix[token] = word2vec.wv.__getitem__(word)
  return embedding_matrix


## Glove vectors

In [ ]:
def get_glove():
  # get_processed_data()
  Glove_dict= {}
  with open('glove.6B.300d.txt', encoding='utf8') as f:
    for line in f:
          values = line.split()
          word = values[0]
          vector = np.asarray(values[1:], "float32")
          Glove_dict[word] = vector
    zeros = np.zeros(300)
    X_train_glove = [ np.sum([ Glove_dict.get(x, zeros) for x in X_train[i] ], axis=0)/len(X_train[i])  for i in range(len(X_train)) ]
    X_test_glove =  [ np.sum([ Glove_dict.get(x, zeros) for x in X_test[i] ], axis=0)/len(X_test[i])   for i in range(len(X_test)) ]
    return np.array(X_train_glove), np.array(X_test_glove), Glove_dict


def get_glove_seq(vocab):
  embedding_matrix = np.zeros((len(vocab)+1, 300))
  Glove_dict= {}
  with open('glove.6B.300d.txt', encoding='utf8') as f:
    for line in f:
          values = line.split()
          word = values[0]
          vector = np.asarray(values[1:], "float32")
          Glove_dict[word] = vector
  zeros = np.zeros(300)
  for word, token in vocab.items():
    embedding_matrix[token] = Glove_dict.get(word, zeros)
  return embedding_matrix


## TF-IDF Vectorizer 

In [ ]:
def get_tfidf():
  # tweets should be untokenized
  get_processed_data(tokenized = False)
   
  # initialize vectorizer 
  tfIdfVectorizer=TfidfVectorizer(lowercase=False)

  # we fit the training data and we transform the test data 
  X_train_tfidf = tfIdfVectorizer.fit_transform(X_train)
  X_test_tfidf = tfIdfVectorizer.transform(X_test)

  return X_train_tfidf, X_test_tfidf, TfidfVectorizer

## One hot encoding

In [ ]:
def seq_data():
  get_processed_data()
  tokenizer = Tokenizer(oov_token="<oov>", document_count=0)
  tokenizer.fit_on_texts(X_train,)
  vocab_size = len(tokenizer.word_index) + 1

  X_train_pad = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=60)

  X_test_pad = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=60)

  return X_train_pad, X_test_pad, tokenizer

In [ ]:
# this code to save the trained tokenizer and embeddings

with open('embeddings_matrices.pickle', 'wb') as embed:
    pickle.dump(embeddings_matrices, embed, protocol=pickle.HIGHEST_PROTOCOL)

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

## ELMo

In [ ]:
def elmo_embed(x):
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))
    

In [ ]:
def get_elmo():

  # tr_path = pathlib.Path('/content/drive/MyDrive/X_train_elmo.pickle')
  # te_path = pathlib.Path('/content/drive/MyDrive/X_test_elmo.pickle')
  
  # # if files exists we load them instead of training from scratch 
  # if tr_path.is_file() and te_path.is_file():
  #   pickle_in = open(tr_path, "rb")
  #   X_elmo_train = pickle.load(pickle_in)
  #   pickle_in = open(te_path, "rb")
  #   X_elmo_test = pickle.load(pickle_in)
  #   return X_elmo_train, X_elmo_test

  # get processed and untokenized data,
  X_train, X_test = get_processed_data(False)

  # to speed up transforming words into elmo vectors it's recommanded to transform batches instead of one at a go 
  X_elmo_train = [X_train[i:i+100] for i in range(0,X_train.shape[0],100)]
  X_elmo_test = [X_test[i:i+100] for i in range(0,X_test.shape[0],100)]

  # here we start getting the batches embeddings 
  X_elmo_train = [elmo_embed(x) for x in X_elmo_train]
  X_elmo_test = [elmo_embed(x) for x in X_elmo_test]

  # now we concatenate back the vectors 
  X_elmo_train = np.concatenate(X_elmo_train, axis = 0)
  X_elmo_test = np.concatenate(X_elmo_test, axis = 0)
  # we save the vectors as pickle files
  pickle_out = open("X_train_elmo.pickle","wb")
  pickle.dump(X_elmo_train, pickle_out)
  pickle_out.close()

  pickle_out = open("X_test_elmo.pickle","wb")
  pickle.dump(X_elmo_test, pickle_out)
  pickle_out.close()

  return X_elmo_train, X_elmo_test

## load data and embeddings 

In [ ]:
def get_embeddings():
  get_processed_data()
  embeddings = {
                'glove': (get_glove()),
                'w2v': (get_w2v()),
                'bert':(get_bert())
                }
  get_processed_data(False)
  embeddings['tfidf'] = get_tfidf()
  return embeddings

In [ ]:
# embeddings = get_embeddings()
# with open('embeddings.pickle', 'wb') as embed:
#     pickle.dump(embeddings, embed, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
def get_seq_embeddings(vocab):
  embeddings_matrices = {
                'glove': (get_glove_seq(vocab)),
                'w2v': (get_w2v_seq(vocab)),
                'bert':(get_bert_Seq(vocab))
                }
  return embeddings_matrices

# Models and architectures 


##RNN based models

### architectures

In [ ]:
X_train_pad, X_test_pad, tokenizer = seq_data()
# embeddings_matrices = get_seq_embeddings(tokenizer.word_index)


In [ ]:
def lstm(embedding_matrix):
  embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[embedding_matrix], input_length=60)
  #  LSTM model with 100   
  model = Sequential()
  model.add(embedding_layer)
  model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model_history=model.fit(X_train_pad, Y_train,batch_size=1024,epochs=3, validation_split=0.1, verbose=1)
  scores = model.predict(X_test_pad)
  y_pred=np.where(scores>0.5,1,0)
  accu = accuracy_score(Y_test, y_pred)
  print(accu)
  return accu, model

In [ ]:
def Bidirectional_lstm(embedding_matrix):
  input_shape = embedding_matrix.shape[0], embedding_matrix.shape[1]
  embedding_layer = Embedding(input_shape[0], input_shape[1], weights=[embedding_matrix], input_length=60)
  #  LSTM model with 100   
  # Two stacked Bidirectional LSTM
  model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(embedding_matrix.shape[1], return_sequences=True), input_shape=input_shape),
        Bidirectional(LSTM(60)),
        Dense(1, activation='sigmoid'),
    ])
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model_history=model.fit(X_train_pad, Y_train,batch_size=1024,epochs=3, validation_split=0.1, verbose=1)
  scores = model.predict(X_test_pad)
  y_pred=np.where(scores>0.5,1,0)
  accu = accuracy_score(Y_test, y_pred)
  print(accu)
  return accu

In [ ]:
def mixed_lstm( embedding_matrix):
  embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[embedding_matrix], input_length=60)
  #  LSTM model with 100   
  model = Sequential()
  model.add(embedding_layer)
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(LSTM(100))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model_history=model.fit(X_train_pad, Y_train,batch_size=1024,epochs=3, validation_split=0.1, verbose=1)
  scores = model.predict(X_test_pad)
  y_pred=np.where(scores>0.5,1,0)
  accu = accuracy_score(Y_test, y_pred)
  print(accu)
  return accu

In [ ]:
def dropout_lstm(embedding_matrix):
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[embedding_matrix], input_length=60)
    #  LSTM model with 100   
    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(LSTM(60)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_history=model.fit(X_train_pad, Y_train,batch_size=1024,epochs=3, validation_split=0.1, verbose=1)
    scores = model.predict(X_test_pad)
    y_pred=np.where(scores>0.5,1,0)
    accu = accuracy_score(Y_test, y_pred)
    print(accu)
    return accu


In [ ]:
def Gru(embedding_matrix):
  embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[embedding_matrix], input_length=60)
  model = Sequential([
        embedding_layer,
        GRU(60),
        Dense(1, activation='sigmoid'),
    ])
  model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])
  model_history=model.fit(X_train_pad, Y_train,batch_size=1024,epochs=3, validation_split=0.1, verbose=1)
  scores = model.predict(X_test_pad)
  y_pred=np.where(scores>0.5,1,0)
  accu = accuracy_score(Y_test, y_pred)
  print(accu)
  return accu

### predicting

In [ ]:
# Global RNN based models scores
R_scores = {}

In [ ]:

def lstm_scores(embedding_matrix):
  lstm_score = {}
  for name, embed in embedding_matrix.items():
    if (name == 'bert'):
      lstm_score[name], model = lstm(embed)
  return lstm_score, model
lstm_score, model = lstm_scores(embeddings_matrices)

Epoch 1/3
120/120 [==============================] - 127s 1s/step - loss: 0.6448 - accuracy: 0.6257 - val_loss: 0.5047 - val_accuracy: 0.7500
Epoch 2/3
120/120 [==============================] - 122s 1s/step - loss: 0.4831 - accuracy: 0.7662 - val_loss: 0.4708 - val_accuracy: 0.7700
Epoch 3/3
120/120 [==============================] - 122s 1s/step - loss: 0.4175 - accuracy: 0.8115 - val_loss: 0.4878 - val_accuracy: 0.7664
0.7607083333333333


In [ ]:
lstm_score

{'bert': 0.769, 'glove': 0.7574583333333333, 'w2v': 0.7624166666666666}

In [ ]:
R_scores['lstm'] = lstm_score

In [ ]:
def mixed_lstm_scores(embedding_matrix):
  lstm_score = {}
  for name, embed in embedding_matrix.items():
    lstm_score[name] = mixed_lstm(embed)
  return lstm_score
mixed_lstm_score = mixed_lstm_scores(embeddings_matrices)

Epoch 1/3
120/120 [==============================] - 45s 301ms/step - loss: 0.6071 - accuracy: 0.6588 - val_loss: 0.4929 - val_accuracy: 0.7568
Epoch 2/3
120/120 [==============================] - 35s 291ms/step - loss: 0.4464 - accuracy: 0.7958 - val_loss: 0.4801 - val_accuracy: 0.7668
Epoch 3/3
120/120 [==============================] - 35s 293ms/step - loss: 0.3438 - accuracy: 0.8547 - val_loss: 0.5227 - val_accuracy: 0.7565
0.752625
Epoch 1/3
120/120 [==============================] - 15s 114ms/step - loss: 0.5926 - accuracy: 0.6855 - val_loss: 0.5022 - val_accuracy: 0.7507
Epoch 2/3
120/120 [==============================] - 13s 109ms/step - loss: 0.4758 - accuracy: 0.7739 - val_loss: 0.4754 - val_accuracy: 0.7724
Epoch 3/3
120/120 [==============================] - 13s 107ms/step - loss: 0.3913 - accuracy: 0.8272 - val_loss: 0.5119 - val_accuracy: 0.7613
0.7527083333333333
Epoch 1/3
120/120 [==============================] - 98s 796ms/step - loss: 0.6067 - accuracy: 0.6575 - val_

In [ ]:
mixed_lstm_score 

{'bert': 0.765375, 'glove': 0.752625, 'w2v': 0.7527083333333333}

In [ ]:
R_scores['mixed_lstm'] = mixed_lstm_score

In [ ]:
def dropout_lstm_scores(embedding_matrix):
  lstm_score = {}
  for name, embed in embedding_matrix.items():
    lstm_score[name] = dropout_lstm(embed)
  return lstm_score
dropout_lstm_score = dropout_lstm_scores(embeddings_matrices)

Epoch 1/3
120/120 [==============================] - 61s 432ms/step - loss: 0.6073 - accuracy: 0.6618 - val_loss: 0.4977 - val_accuracy: 0.7566
Epoch 2/3
120/120 [==============================] - 50s 418ms/step - loss: 0.4626 - accuracy: 0.7882 - val_loss: 0.4795 - val_accuracy: 0.7697
Epoch 3/3
120/120 [==============================] - 49s 411ms/step - loss: 0.3837 - accuracy: 0.8345 - val_loss: 0.5152 - val_accuracy: 0.7550
0.7499583333333333
Epoch 1/3
120/120 [==============================] - 25s 179ms/step - loss: 0.6048 - accuracy: 0.6635 - val_loss: 0.5045 - val_accuracy: 0.7547
Epoch 2/3
120/120 [==============================] - 22s 181ms/step - loss: 0.4812 - accuracy: 0.7715 - val_loss: 0.4751 - val_accuracy: 0.7696
Epoch 3/3
120/120 [==============================] - 21s 172ms/step - loss: 0.4108 - accuracy: 0.8175 - val_loss: 0.4919 - val_accuracy: 0.7651
0.7582916666666667
Epoch 1/3
120/120 [==============================] - 114s 916ms/step - loss: 0.6265 - accuracy: 0.

In [ ]:
dropout_lstm_score

{'bert': 0.7545833333333334,
 'glove': 0.7499583333333333,
 'w2v': 0.7582916666666667}

In [ ]:
R_scores['dropout_lstm'] = dropout_lstm_score

In [ ]:
def Bidirectional_lstm_scores(embedding_matrix):
  bidirectional_score = {}
  for name, embed in embedding_matrix.items():
    bidirectional_score[name] = Bidirectional_lstm(embed)
  return bidirectional_score
bidirectional_score = Bidirectional_lstm_scores(embeddings_matrices)


Epoch 1/3
120/120 [==============================] - 106s 828ms/step - loss: 0.5761 - accuracy: 0.6889 - val_loss: 0.4849 - val_accuracy: 0.7625
Epoch 2/3
120/120 [==============================] - 98s 816ms/step - loss: 0.4391 - accuracy: 0.7970 - val_loss: 0.4797 - val_accuracy: 0.7688
Epoch 3/3
120/120 [==============================] - 97s 813ms/step - loss: 0.3382 - accuracy: 0.8549 - val_loss: 0.5426 - val_accuracy: 0.7525
0.7517916666666666
Epoch 1/3
120/120 [==============================] - 42s 291ms/step - loss: 0.5733 - accuracy: 0.6983 - val_loss: 0.5014 - val_accuracy: 0.7555
Epoch 2/3
120/120 [==============================] - 33s 279ms/step - loss: 0.4570 - accuracy: 0.7874 - val_loss: 0.4826 - val_accuracy: 0.7656
Epoch 3/3
120/120 [==============================] - 34s 281ms/step - loss: 0.3542 - accuracy: 0.8479 - val_loss: 0.5091 - val_accuracy: 0.7603
0.7545
Epoch 1/3
120/120 [==============================] - 339s 3s/step - loss: 0.6953 - accuracy: 0.5785 - val_los

In [ ]:
bidirectional_score

{'bert': 0.7671666666666667, 'glove': 0.7517916666666666, 'w2v': 0.7545}

In [ ]:
R_scores['bidirectional_lstm'] = bidirectional_score

In [ ]:
def gru_scores(embedding_matrix):
  gru_score = {}
  for name, embed in embedding_matrix.items():
    gru_score[name] = Gru(embed)
  return gru_score
gru_score = gru_scores(embeddings_matrices)


Epoch 1/3
120/120 [==============================] - 49s 388ms/step - loss: 0.5966 - accuracy: 0.6676 - val_loss: 0.4893 - val_accuracy: 0.7621
Epoch 2/3
120/120 [==============================] - 47s 388ms/step - loss: 0.4385 - accuracy: 0.7998 - val_loss: 0.4857 - val_accuracy: 0.7671
Epoch 3/3
120/120 [==============================] - 46s 384ms/step - loss: 0.3424 - accuracy: 0.8552 - val_loss: 0.5407 - val_accuracy: 0.7489
0.74325
Epoch 1/3
120/120 [==============================] - 20s 154ms/step - loss: 0.6050 - accuracy: 0.6568 - val_loss: 0.4986 - val_accuracy: 0.7584
Epoch 2/3
120/120 [==============================] - 17s 145ms/step - loss: 0.4688 - accuracy: 0.7784 - val_loss: 0.4725 - val_accuracy: 0.7751
Epoch 3/3
120/120 [==============================] - 18s 146ms/step - loss: 0.3579 - accuracy: 0.8481 - val_loss: 0.5379 - val_accuracy: 0.7524
0.7454583333333333
Epoch 1/3
120/120 [==============================] - 102s 829ms/step - loss: 0.6224 - accuracy: 0.6412 - val_

In [ ]:
gru_score

{'bert': 0.7592083333333334, 'glove': 0.74325, 'w2v': 0.7454583333333333}

In [ ]:
R_scores['gru'] = gru_score

In [ ]:
for key in R_scores.keys():
  R_scores[key]['tfidf'] = '--'

## Classic algorithms

### algorithms

In [ ]:
def logistic_model(X_tr, y_tr, X_test, y_test, description):
    clf = LogisticRegression(max_iter=1000).fit(X_tr, y_tr)
    score = clf.score(X_test, y_test)
    return score, clf

In [ ]:
def svm_model(X_train, Y_train, X_test, Y_test, description):
   clf = LinearSVC(random_state=0, tol=1e-5)
   clf.fit(X_train, Y_train)
   score = clf.score(X_test, Y_test)
   return score, clf

In [ ]:
def AdaBoost_model(X_tr, y_tr, X_test, y_test, description):
  clf = AdaBoostClassifier(n_estimators=100, random_state=0)
  clf.fit(X_tr, y_tr)
  score = clf.score(X_test, y_test)
  return score, clf
  

In [ ]:
def Xgboost_model(X_tr, y_tr, X_test, y_test, description):
  clf = XGBClassifier(random_state=0)
  clf.fit(X_tr, y_tr)
  score = clf.score(X_test, y_test)
  return score, clf 


### predicting

#### lg_Test

In [ ]:
def get_lg_scores(embeddings):
  scores = []
  lg_models = []
  for key,value in embeddings.items():
    score, model = logistic_model(value[0], Y_train, value[1], Y_test, key)
    scores.append(score)
    lg_models.append(model)
  return scores, lg_models

In [ ]:
lg_score, lg_models = get_lg_scores(embeddings)

In [ ]:
scores = { "logistic_regression": lg_score}

#### SVM_test

In [ ]:
def get_svm_scores(embeddings):
  scores = []
  svm_models = []
  for key,value in embeddings.items():
    score, model = svm_model(value[0], Y_train, value[1], Y_test, key)
    scores.append(score)
    # svm_models.append[model]
  return scores,svm_models

In [ ]:
svm_scores, svm_models = get_svm_scores(embeddings)

In [ ]:
scores["svm"] = svm_scores

#### AdaBoost

In [ ]:
def get_AdaBoost_scores(embeddings):
  scores = []
  ada_models = []
  for key,value in embeddings.items():
    score, model = AdaBoost_model(value[0], Y_train, value[1], Y_test, key)
    scores.append(score)
    # ada_models.append(model)
  return scores, ada_models

In [ ]:

AdaBoost_scores, ada_models = get_AdaBoost_scores(embeddings)

In [ ]:
scores["AdaBoost"] = AdaBoost_scores

#### XGBoost

In [ ]:
def get_XGBoost_scores(embeddings):
  scores = []
  XGB_models = []
  for key,value in embeddings.items():
    score, model = Xgboost_model(value[0], Y_train, value[1], Y_test, key)
    scores.append(score)
    # XGB_models.append(model)
  return scores 

In [ ]:
XGBoost_scores = get_XGBoost_scores(embeddings)

In [ ]:
scores["XGBoost"] = XGBoost_scores

# results

In [ ]:
results = pd.DataFrame.from_dict(scores, orient='index',columns=[key for key in embeddings.keys()])

In [ ]:
results

,glove,w2v,bert,tfidf
logistic_regression,0.698625,0.728417,0.721500,0.765167
svm,0.699125,0.728917,0.718125,0.751333
AdaBoost,0.667958,0.715958,0.686125,0.704125
XGBoost,0.681333,0.721583,0.693333,0.680167


In [ ]:
R_results = pd.DataFrame.from_dict(R_scores, orient='index')

In [ ]:
R_results

,bert,glove,w2v,tfidf
lstm,0.769000,0.757458,0.762417,--
mixed_lstm,0.765375,0.752625,0.752708,--
dropout_lstm,0.754583,0.749958,0.758292,--
bidirectional_lstm,0.767167,0.751792,0.754500,--
gru,0.759208,0.743250,0.745458,--
